In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

import missingno as mgn

In [ ]:
df = pd.read_excel('../data/discover_2024-03-26.xlsx', sheet_name='data')
df.columns = [col.lower() for col in df.columns]

In [ ]:
df.info()

In [ ]:
titles = pd.read_csv('../data/real_title.csv', header=None, names=['real_title'])
titles # the order is the same as in the original dataframe!

In [ ]:
old_df = pd.read_excel('../data/discover_0123_0324.xlsx', sheet_name='data')
old_df.columns = [col.lower() for col in old_df.columns]
old_df.head()

In [ ]:
old_df = pd.concat((old_df, titles), axis=1)

In [ ]:
old_df_ = old_df[['page_efahrer_id', 'date', 'page_canonical_url', 'real_title']]

In [ ]:
df_merged = pd.merge(left=df, right=old_df_, on=['page_efahrer_id', 'date', 'page_canonical_url'], how='left')
df_merged

In [ ]:
df.head(20)

The first article looks completely differently nowadays!

In [ ]:
df[df['title'].str.contains("Neueste Förderung")]

In [ ]:
df.set_index('page_efahrer_id').loc[1010803]

WORD_COUNT	VIDEO_PLAY	CLICKOUTS

In [ ]:
mgn.matrix(df.sort_values(['page_efahrer_id', 'date', 'page_name']));

In [ ]:
mgn.dendrogram(df.sort_values(['page_efahrer_id', 'date', 'page_name']));

In [ ]:
mgn.heatmap(df)

In [ ]:
df.isna().sum()

In [ ]:
counts = df.groupby(["page_efahrer_id"]).count()
counts.query("title > 2").iloc[:, 0]

## Check uniqness of the attribute

In [ ]:
df[["page_efahrer_id", "classification_product", "classification_type"]].drop_duplicates()\
    .groupby("page_efahrer_id").count().max() #.value_counts()

In [ ]:
df.classification_product.unique()

In [ ]:
df.classification_type.unique()

In [ ]:
df.set_index("page_efahrer_id").loc[1018632] #.PAGE_AUTHOR.unique()

### `WORD_COUNT`

In [ ]:
df.word_count.describe()

In [ ]:
df.word_count.hist(bins=50);

looks like no anomalies

In [ ]:
df[["page_canonical_url", "word_count"]].query("word_count > 3000")

In [ ]:
df[["page_efahrer_id", "word_count"]].sort_values("page_efahrer_id").tail(30)

In [ ]:
df[df.word_count.isna() & ~df.published_at.isna()]

In [ ]:
art_unique_wcnts = df.value_counts(["page_efahrer_id", "word_count"])
art_unique_wcnts.to_frame().rename({'count': 'n_article_occurances'}, axis=1)

In [ ]:
art_unique_wcnts = df.value_counts(["page_efahrer_id", "word_count"])
art_unique_wcnts = art_unique_wcnts.to_frame().rename({'count': 'n_article_occurances'}, axis=1)
art_unique_wcnts.reset_index() 

In [ ]:
art_unique_wcnts

In [ ]:
diff_word_counts = art_unique_wcnts.to_frame().reset_index().value_counts() # .to_frame()

In [ ]:
diff_word_counts

In [ ]:
df.nunique()

### `VIDEO_PLAY`

In [ ]:
df[df.video_play.isna()].isna().sum()

In [ ]:
df.video_play.describe()

In [ ]:
df.query("video_play >= 700000")

In [ ]:
df.shape

In [ ]:
print(f"""
      Broken:  {df.query("video_play >= 703621").shape}, 
      Normal:  {df.query("video_play < 703621").shape}, 
      Missing: {df[df.video_play.isna()].shape}
""")

In [ ]:
import seaborn as sns
import datetime
now = datetime.datetime.now()
sns.scatterplot(x=pd.to_timedelta(now - df["date"]).dt.days, y=df["video_play"])

---

In [ ]:
# unique ids

scraping = df[['page_efahrer_id', 'page_canonical_url']].drop_duplicates()
scraping

In [ ]:
scraping = scraping.to_frame()

# creating the dummy columns that will be populated with the scraped data
scraping['H1'] = 'dummy'
scraping['last_update_date'] = 'today'
scraping['abstract'] = 'null'

scraping = scraping.set_index('page_efahrer_id')

scraping

In [ ]:
## Do actual scraping

i=0
for row_idx in scraping.index:
    scraping.loc[row_idx, 'H1'] = ##the header
    scraping.loc[row_idx, 'last_update_date'] = ##the last update time element
    scraping.loc[row_idx, 'abstract'] = ##find a clever way to extract the abstract?
    i+=1
    if i==10: 
        scraping.to_csv('../data/temp_scraped.csv')
        i=0